In [ ]:
import cv2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical

import cv2
import numpy as np
import time


# ignore//to isolate the hand region

In [ ]:

cam = cv2.VideoCapture(0)
time.sleep(2)
i = 0
while True:
    ret,frame = cam.read()
    #cv2.imshow('webcam', frame)
    cv2.imwrite('captures/'+str(i)+'.jpg',frame)
    i+=1
    if cv2.waitKey(1)&0xFF == ord('q'):
        break

In [ ]:
cam.release()
cv2.destroyAllWindows(

In [ ]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

In [ ]:
#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [ ]:
#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (_, cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [ ]:
#-----------------
# MAIN FUNCTION
#-----------------
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                #cv2.imshow("Thesholded", thresholded)
                cv2.imwrite('captures/'+str(num_frames)+'.jpg',thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)
        

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory


In [ ]:
camera.release()
cv2.destroyAllWindows()

# live video capture with mediapipe

In [ ]:
import mediapipe as mp

In [ ]:
pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


In [ ]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
            continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

    # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break


In [ ]:
cap.release()

# preprocess the dataset

In [ ]:
from IPython.display import Image
Image("asl/amer_sign2.png")

In [ ]:
# Read Dataset
train = pd.read_csv('asl/sign_mnist_train.csv')
test = pd.read_csv('asl/sign_mnist_test.csv')

In [ ]:
train.head()

In [ ]:
labels = train['label']

In [ ]:
# droping the label coloumn from the training set
train.drop('label', axis = 1, inplace = True)
print(train.shape)

In [ ]:
x_train = train.values.reshape(train.shape[0],28,28,1)

In [ ]:
x_train.shape

In [ ]:
plt.imshow(x_train[0].reshape(28,28), cmap='gray')

In [ ]:
plt.imsave("cropped/0.png",x_train[0].reshape(28,28),cmap='gray')

In [ ]:
def crop(x_train):
    for i in range(len(x_train)):
        plt.imsave("cropped/cropped"+str(i)+".png",x_train[i].reshape(28,28),cmap='gray')

# images belong to the same folder 

In [ ]:
import os
os.getcwd()

In [ ]:
# loop thru all the files in a folder 
filenames = []
directory = '/Users/bridgetliu/project/mais2021/archive/asl_alphabet_train/asl_alphabet_train/K'
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        filenames.append(os.path.join(directory, filename))
    else:
        continue

In [ ]:
filenames

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

IMAGE_FILES = filenames
#IMAGE_FILES = ["archive/asl_alphabet_train/asl_alphabet_train/K/K294.jpg"]
#IMAGE_FILES = ["test.jpg"]
all_img_features = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    for idx, file in enumerate(IMAGE_FILES):
        print("here")
        featurelist = []
        # Read an image, flip it around y-axis for correct handedness output
        image = cv2.flip(cv2.imread(file), 1)
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #results = hands.process(image)
        #Print handedness and draw hand landmarks on the image.
        featurelist.append('k')
        featurelist.append(results.multi_handedness)
        print('Handedness:', results.multi_handedness)
        if not results.multi_hand_landmarks:
            continue
        
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        
        for hand_landmarks in results.multi_hand_landmarks:
            featurelist.append(hand_landmarks)
            print('hand_landmarks:', hand_landmarks)
            print(
               f'Index finger tip coordinates: (',
               f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
               f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
            )
            '''
            mp_drawing.draw_landmarks(
               annotated_image,
               hand_landmarks,
               mp_hands.HAND_CONNECTIONS,
               mp_drawing_styles.get_default_hand_landmarks_style(),
               mp_drawing_styles.get_default_hand_connections_style())
            '''
        #cv2.imwrite(
        #'/Users/bridgetliu/project/mais2021/tmp'+str(idx)+'.png', cv2.flip(annotated_image, 1))
        all_img_features.append(featurelist)


In [ ]:
featurelist

In [ ]:
import numpy as np
from numpy import savetxt

In [ ]:
tmp = np.array(all_img_features,dtype = object)
savetxt('k.csv', tmp,fmt='%s')

In [ ]:
tmp[0]

# for multiple folders

In [ ]:
directory = '/Users/bridgetliu/project/mais2021/archive/asl_alphabet_train/asl_alphabet_train/'
count = 0 
names = []
folders = []
for root, subdirectories, files in os.walk(directory):
    for subdirectory in subdirectories:
        count +=1
        #print(os.path.join(root, subdirectory))
        # loop thru all the files in a folder 
        filenames = []
        directory = os.path.join(root, subdirectory)
        for filename in os.listdir(directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                filenames.append(os.path.join(directory, filename))
            else:
                continue
        names.append(filenames)
        folders.append(subdirectory)

In [ ]:
len(names[0])

In [ ]:
count

In [ ]:
foldersubdirectory in subdirectories:

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands



for i in range(29):
    IMAGE_FILES = names[i]
    all_img_features = []
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        for idx, file in enumerate(IMAGE_FILES):
            print("here")
            featurelist = []
            # Read an image, flip it around y-axis for correct handedness output
            image = cv2.flip(cv2.imread(file), 1)
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            #results = hands.process(image)
            #Print handedness and draw hand landmarks on the image.
            featurelist.append(folders[i])
            featurelist.append(results.multi_handedness)
            print('Handedness:', results.multi_handedness)
            if not results.multi_hand_landmarks:
                continue
        
            image_height, image_width, _ = image.shape
            annotated_image = image.copy()
        
            for hand_landmarks in results.multi_hand_landmarks:
                featurelist.append(hand_landmarks)
                print('hand_landmarks:', hand_landmarks)
                print(
                   f'Index finger tip coordinates: (',
                   f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                   f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                )
            '''
            mp_drawing.draw_landmarks(
               annotated_image,
               hand_landmarks,
               mp_hands.HAND_CONNECTIONS,
               mp_drawing_styles.get_default_hand_landmarks_style(),
               mp_drawing_styles.get_default_hand_connections_style())
            '''
            #cv2.imwrite(
            #'/Users/bridgetliu/project/mais2021/tmp'+str(idx)+'.png', cv2.flip(annotated_image, 1))
            all_img_features.append(featurelist)


In [ ]:
tmp = np.array(all_img_features,dtype = object)
savetxt('features.csv', tmp,fmt='%s')